<a href="https://colab.research.google.com/github/PhiDCH/TextFuseNet/blob/master/TextFuseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluate ICDAR_2015

## prepare data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir /content/icdar2015
!mkdir /content/icdar2015/train_img
!mkdir /content/icdar2015/train_gt
!mkdir /content/icdar2015/test_img
!mkdir /content/icdar2015/test_gt
!unzip /content/drive/MyDrive/MyComputer/icdar2015/ch4_training_images.zip -d /content/icdar2015/train_img
!unzip /content/drive/MyDrive/MyComputer/icdar2015/ch4_training_localization_transcription_gt.zip -d /content/icdar2015/train_gt
!unzip /content/drive/MyDrive/MyComputer/icdar2015/ch4_test_images.zip -d /content/icdar2015/test_img
!unzip /content/drive/MyDrive/MyComputer/icdar2015/Challenge4_Test_Task1_GT.zip -d /content/icdar2015/test_gt

## clone repo and download pretrained model

In [1]:
!rm -rf /content/textfusenet
!git clone https://github.com/PhiDCH/TextFuseNet /content/textfusenet

Cloning into '/content/textfusenet'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 641 (delta 13), reused 0 (delta 0), pack-reused 597
Receiving objects: 100% (641/641), 4.68 MiB | 19.17 MiB/s, done.
Resolving deltas: 100% (191/191), done.


In [3]:
cd /content/textfusenet

/content/textfusenet


In [4]:
!gdown --id 1OWQUxaHpEoWoC6TzArgsz3aVvkJhJhej -O ./pretrain/textfuse-ic15.pth

Downloading...
From: https://drive.google.com/uc?id=1OWQUxaHpEoWoC6TzArgsz3aVvkJhJhej
To: /content/textfusenet/pretrain/textfuse-ic15.pth
915MB [00:10, 87.4MB/s]


In [5]:
!pip install torch==1.3.1+cu100 torchvision==0.4.2+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install opencv-python tensorboard yacs tqdm termcolor tabulate matplotlib cloudpickle wheel pycocotools
!python -m pip install Pillow==6.2
!pip install fvcore-master.zip
!python setup.py build develop

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 705.4MB 26kB/s 
     |████████████████████████████████| 10.2MB 61.7MB/s 
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.3.1+cu100 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:
      Successfully uninstalled torchvision-0.9.0+cu101
     |████████████████████████████████| 2.1MB 8.7MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0
Processing ./fvcore-master.zip
     |████████████████████████████████| 645kB 7.9MB/s 
  Created wheel for fvcore: filename=fvcore

In [10]:
!pip install Polygon3

  Created wheel for Polygon3: filename=Polygon3-3.0.9.1-cp37-cp37m-linux_x86_64.whl size=102604 sha256=73b141e63461d61f3a17947d563dbe97adcd2010ad885ad6632e9ea8ffff5165
  Stored in directory: /root/.cache/pip/wheels/d8/20/1d/52982f2c8dd3794845145b221fe006028e16aa5d80646136ff
Successfully built Polygon3


## test single image

In [10]:
!python demo/demo.py --config-file configs/ocr/icdar2015_101_FPN.yaml \
--input test_img/sample.jpg \
--output result

[03/12 03:59:21 detectron2]: Arguments: Namespace(confidence_threshold=0.5, config_file='configs/ocr/icdar2015_101_FPN.yaml', input=['test_img/sample.jpg'], opts=[], output='result', video_input=None, webcam=False)
WARNING [03/12 03:59:21 d2.config.compat]: Config 'configs/ocr/icdar2015_101_FPN.yaml' has no VERSION. Assuming it to be compatible with latest v2.
  0% 0/1 [00:00<?, ?it/s][03/12 03:59:27 detectron2]: test_img/sample.jpg: detected 100 instances in 1.65s
100% 1/1 [00:01<00:00,  1.86s/it]


## eval icdar2015

In [6]:
!mkdir result_ic15

In [12]:
!python demo/icdar2015_detection.py --help

usage: icdar2015_detection.py [-h] [--config-file FILE] [--weights pth]
                              [--input INPUT] [--output OUTPUT]
                              [--gt_path GT_PATH]
                              [--confidence-threshold CONFIDENCE_THRESHOLD]
                              [--opts ...]

Detectron2 Demo

optional arguments:
  -h, --help            show this help message and exit
  --config-file FILE    path to config file
  --weights pth         the model used to inference
  --input INPUT         the folder of icdar2015 test images
  --output OUTPUT       A file or directory to save output visualizations. If
                        not given, will show output in an OpenCV window.
  --gt_path GT_PATH     the folder of icdar2015 test ground truth
  --confidence-threshold CONFIDENCE_THRESHOLD
                        Minimum score for instance predictions to be shown
  --opts ...            Modify config options using the command-line 'KEY
                        VALUE' pa

In [9]:
!python demo/icdar2015_detection.py --config-file configs/ocr/icdar2015_101_FPN.yaml \
--weights pretrain/textfuse-ic15.pth \
--input ../icdar2015/test_img \
--output result_ic15 \
--gt_path ../icdar2015/test_gt

Config 'configs/ocr/icdar2015_101_FPN.yaml' has no VERSION. Assuming it to be compatible with latest v2.
100% 500/500 [13:59<00:00,  1.68s/it]
Average Time: 1.68 s /img
{'precision': 0.9249502982107356, 'recall': 0.8960038517091959, 'hmean': 0.9102470041574957, 'AP': 0}
